In [154]:
import numpy as np
import pandas as pd
import os
from tika import parser
import re

In [266]:
DATA_PATH = './data/cc_2018/'
files = [f for f in os.listdir(DATA_PATH)]

In [355]:
def _strip(file,path):
    regex = '\n\n(?=\u2028|[A-Z-0-9])'
    try:
        return re.split(regex, 
                            [parser.from_file(path + file + '/' + doc) for doc in os.listdir(path + file)]
                    [0]['content'].strip()
                )
    except:
        return ['no data']

In [356]:
documents = {f: strip_(f,DATA_PATH) for f in files}

In [357]:
len(documents.keys())

651

In [354]:
t_ = {d: [t.replace('\n', ' ').strip() for t in documents[d]] for d in documents.keys()}

{'Greater Niles Community FCU': ['no data']}

In [274]:
clean.keys()

dict_keys(['IBMSECU', 'USE Credit Union', 'Interra Credit Union', 'AMOCO Federal Credit Union', 'Financial Partners Credit Union', 'Vons Employees FCU', 'Vantage Credit Union', 'Tendto Credit Union'])

In [293]:
t_ = []
for f in files[8:9]:
    for doc in os.listdir(DATA_PATH + f):
        t_.append(parser.from_file(DATA_PATH + f + '/' + doc))

In [294]:
t_[0]['content']